In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [2]:
x = sp.symbols('x')
cap = 2000 # total emission cap
Regulator6 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator6)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator6)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator6)
country1 = Country('DE', 1, regulator= Regulator6)
country2 = Country('FI', 0.5, regulator= Regulator6)
country3 = Country('GR', size= 0.1, regulator= Regulator6)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 10*x+ 2*x**2 + 0.1*x**3 , 0, 0, 0, regulator= Regulator6)
firm2 = Firm('firm2', 1, 2, x*0, 11*x+ 3*x**2 + 0.2*x**3, 0, 0, 0, regulator= Regulator6)
firm3 = Firm('firm3', 1, 3, x*0, 5*x+ 4*x**2 + 5*x**3 , 0, 0, 0, regulator= Regulator6)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator6)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator6)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator6)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator6)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator6)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator6)



In [3]:
# Regulator6.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision=0.01)
Regulator6.optimization_with_least_squares(gurobi_print= False, BAU = True)
# Regulator6.optimization_with_least_squares(gurobi_print= False, BAU = False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Optimal solution found
Firm firm1 has output 0 and emission 0
Firm firm2 has output 0 and emission 0
Firm firm3 has output 0 and emission 0
Firm firm4 has output 0 and emission 0
Firm firm5 has output 0 and emission 0
Firm firm6 has output 0 and emission 0
Firm firm7 has output 0 and emission 0
Firm firm8 has output 0 and emission 0
Firm firm9 has output 0 and emission 0
Permit price: 0.0


<gurobi.Model MIP instance Least Squares: 22 constrs, 51 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [4]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp01_varying_free_alloc_of_sector_1_at_90_per_cent_BAU.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator6.emission_cap = 0.9*Regulator6.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator6.sector_registry[1].free_emission_multiplier = i
    Regulator6.optimization_with_least_squares(gurobi_print= False)
    caps2.append(Regulator6.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator6.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator6.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator6.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator6.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp01_varying_free_alloc_of_sector_1_at_90_per_cent_BAU.csv', index=False)

0it [00:00, ?it/s]


In [5]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp02_varying_free_alloc_of_sector_1_at_80_per_cent_BAU.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator6.emission_cap = 0.8*Regulator6.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator6.sector_registry[1].free_emission_multiplier = i
    Regulator6.optimization_with_least_squares(gurobi_print= False, print_output = False)
    caps2.append(Regulator6.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator6.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator6.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator6.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator6.firm_registry.values()]))
    Regulator6.equilibrium_tester(precision = 0.1, full_output=False, output = True)
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp02_varying_free_alloc_of_sector_1_at_80_per_cent_BAU.csv', index=False)

0it [00:00, ?it/s]


In [6]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp03_varying_free_alloc_of_sector_1_at_70_per_cent_BAU.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator6.emission_cap = 0.7*Regulator6.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):

    Regulator6.sector_registry[1].free_emission_multiplier = i
    Regulator6.optimization_with_least_squares(gurobi_print= True, print_output = False)
    caps2.append(Regulator6.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator6.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator6.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator6.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator6.firm_registry.values()]))
    Regulator6.equilibrium_tester(precision = 0.1, full_output=False, output = True)


    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp03_varying_free_alloc_of_sector_1_at_70_per_cent_BAU.csv', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-9900 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 49 rows, 105 columns and 190 nonzeros
Model fingerprint: 0xb2595080
Model has 3 quadratic constraints
Model has 43 general constraints
Variable types: 105 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-04, 3e+01]
  QMatrix range    [2e-04, 2e-04]
  QLMatrix range   [1e-04, 1e+00]
  Objective range  [1e+00, 2e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]
  QRHS range       [1e+00, 1e+00]
Presolve added 87 rows and 4889 columns
Presolve time: 0.02s
Presolved: 148 rows, 4994 columns, 14915 nonzeros
Presolved model has 43 SOS constraint(s)
Presolved model has 3 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 4994 continuous, 0 integer (0 binary)

Roo

100%|██████████| 1/1 [01:08<00:00, 68.90s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.4974 ⚠️ |   +0.0395 ✔️ |   -9.5676 ✔️ |   -9.7676 ✔️ |   +1.9135 ✔️ | 4/5
